In [1]:
import pandas as pd
import numpy as np

import importlib
import boto3
import requests
import io
import json
from tqdm import tqdm

pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100) 

s3 = boto3.client('s3')
region = boto3.Session().region_name
runtime_client = boto3.client('runtime.sagemaker')

In [2]:
test_df=pd.read_parquet('s3://sofi-data-science/hpeng/pl-gen4/artifact/df_endpoint_testing.parquet')

In [3]:
check_value=pd.DataFrame()

## score testing

In [5]:
for i in tqdm(range(0,10000)):
    
    ### input data***
    test1=test_df[i:i+1]
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-prescreen-20220316', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)
    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    check_value.at[i,'id']=test1['id'].values
    check_value.at[i,'applicant_type']=test1['applicant_type'].values
    check_value.at[i,'score_endpoint_prescreen']= bb['score']

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-underwriting-20220406', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)
    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    check_value.at[i,'score_endpoint_uw']= bb['score']

100%|██████████| 10000/10000 [27:56<00:00,  5.97it/s]


In [6]:
check_value['id'] = check_value['id'].astype(int)
check_value['applicant_type'] = check_value['applicant_type'].str[0]

In [7]:
df_check_combine=pd.merge(check_value,test_df,how='inner',on=['id','applicant_type'])
df_check_combine.shape

(10000, 96)

In [8]:
df_check_combine['prescreen_score_dif'] = df_check_combine['gen4_prescreen_score'] - df_check_combine['score_endpoint_prescreen']
df_check_combine['uw_score_dif'] = df_check_combine['gen4_underwriting_score_29'] - df_check_combine['score_endpoint_uw']

In [9]:
df_check_combine['prescreen_score_dif'].describe()

count    10000.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: prescreen_score_dif, dtype: float64

In [10]:
df_check_combine['uw_score_dif'].describe()

count    10000.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: uw_score_dif, dtype: float64

In [11]:
df_check_combine.to_parquet('s3://sofi-data-science/hpeng/pl-gen4/artifact/df_endpoint_testing_result.parquet')

## test invalid treatment

In [12]:
#prescreen
pr_list = ['p13_bcc5520',
't11_tmti2752',
't11_tstu2752',
'p13_iqz9420',
't11_tall3205',
'p13_bcc8322',
't11_tbca4504',
'p13_upl8132',
't11_tbca3530',
't11_trev0722',
'p13_reh7120',
't11_tbcc1303',
'p13_all8352',
'p13_all7936',
't11_tiln2754',
'p13_iqz9425',
'p13_iqt9510',
't11_tbcc3305',
'p13_iqf9540',
't11_tiln2755',
't11_tbca2526',
'p13_upl0438',
'p13_upl8320',
'p13_all7518',
'p13_aut8140',
'p13_cru8320',
'p13_all7312',
'p13_rta7300',
't11_tbca2381',
'p13_cru1300',
'p13_rtr5520']

for attr in pr_list:
    test1=test_df[0:1].copy()
    test1[attr] = -999999999
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-prescreen-20220316', 
                                                  ContentType='application/json',Accept='application/json', Body=Body_content)
    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

...

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

In [13]:
## member & nonmember
mem_list = ['t11_tbca2526',
't11_tstu1752',
'sofi_all5840_mtf5838_to_income',
't11_tall02q3',
't11_tall3205',
'current_pl_trade_bal_ratio',
'p13_bcc8322',
't11_tpil01q1',
'sofi_num_month_recent_list_psl',
't11_tupl01q1',
't11_tbcc1303',
'p13_iqz9420',
't11_tbca3243',
'sofi_num_inq_12month',
't11_tbcc4355',
't11_tbcc2267',
'p13_mtf8111',
'p13_all8352',
'all_time_months_oldest_pl_trade',
'p13_iqz9426',
'p13_all8800',
't11_tiln2755',
'p13_pil8320',
'p13_all7110',
't11_tmti02q7',
'p13_bcc7580',
't11_tall0722',
'p13_all2919',
't11_tiln0727',
'p13_iqb9740',
'p13_iln8222']
nm_list = [
    'p13_bcc5520',
'sofi_all5840_mtf5838_to_income',
't11_tall2205',
'p13_bca8370',
't11_tstu2752',
'p13_iqz9427',
'p13_upl8132',
't11_tmti2752',
't11_tiln2755',
'sofi_num_inq_3month',
't11_tbca4504',
'sofi_num_inq_personal_finance',
't11_trev0722',
't11_tbcc4208',
'p13_all8352',
'p13_all7936',
'p13_all5320',
't11_tbca2530',
'sofi_num_inq_1month',
'p13_reh7120',
'p13_all7519',
't11_tbca2303',
'p13_bca7600',
'p13_upl8320',
'p13_bcx7110',
'p13_all7312',
'p13_cru8320',
'p13_rta7300',
'p13_rtr5520'
]
for attr in mem_list:
    test1=test_df[0:1].copy()
    test1[attr] = -999999999
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-underwriting-20220406', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)

    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)
    
for attr in nm_list:
    test1=test_df[5000:5001].copy()
    test1[attr] = -999999999
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-underwriting-20220406', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)

    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

...

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

{'score': -99.0, 'version': 'v1', 'error': 'invalid input'}

### test missing treatment

In [14]:
#prescreen
for attr in pr_list:
    test1=test_df[0:1].copy()
    test1[attr] = np.nan
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-prescreen-20220316', 
                                                  ContentType='application/json',Accept='application/json', Body=Body_content)
    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

...

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

In [15]:
## member & nonmember
mem_list = ['t11_tbca2526',
't11_tstu1752',
'sofi_all5840_mtf5838_to_income',
't11_tall02q3',
't11_tall3205',
'current_pl_trade_bal_ratio',
'p13_bcc8322',
't11_tpil01q1',
'sofi_num_month_recent_list_psl',
't11_tupl01q1',
't11_tbcc1303',
'p13_iqz9420',
't11_tbca3243',
'sofi_num_inq_12month',
't11_tbcc4355',
't11_tbcc2267',
'p13_mtf8111',
'p13_all8352',
'all_time_months_oldest_pl_trade',
'p13_iqz9426',
'p13_all8800',
't11_tiln2755',
'p13_pil8320',
'p13_all7110',
't11_tmti02q7',
'p13_bcc7580',
't11_tall0722',
'p13_all2919',
't11_tiln0727',
'p13_iqb9740',
'p13_iln8222']
nm_list = [
    'p13_bcc5520',
'sofi_all5840_mtf5838_to_income',
't11_tall2205',
'p13_bca8370',
't11_tstu2752',
'p13_iqz9427',
'p13_upl8132',
't11_tmti2752',
't11_tiln2755',
'sofi_num_inq_3month',
't11_tbca4504',
'sofi_num_inq_personal_finance',
't11_trev0722',
't11_tbcc4208',
'p13_all8352',
'p13_all7936',
'p13_all5320',
't11_tbca2530',
'sofi_num_inq_1month',
'p13_reh7120',
'p13_all7519',
't11_tbca2303',
'p13_bca7600',
'p13_upl8320',
'p13_bcx7110',
'p13_all7312',
'p13_cru8320',
'p13_rta7300',
'p13_rtr5520'
]
for attr in mem_list:
    test1=test_df[0:1].copy()
    test1[attr] = np.nan
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-underwriting-20220406', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)

    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)
    
for attr in nm_list:
    test1=test_df[5000:5001].copy()
    test1[attr] = np.nan
    Body_content=test1.to_json(orient='records')

    response = runtime_client.invoke_endpoint(EndpointName='data-science-risk-rds-deploy-pl-gen4-underwriting-20220406', 
                                              ContentType='application/json',Accept='application/json', Body=Body_content)

    result = response['Body'].read().decode('ascii')
    bb=json.loads(result)[0]
    display(bb)

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

...

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}

{'score': -88.0, 'version': 'v1', 'error': 'missing input'}